In [1]:
import sys
sys.path.append("..")   
from functions import get_data, data_split_train_val
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Load Fashion-MNIST
X, y = get_data("../data/fashion-mnist_test.csv")   # normalised to 0–1

# Split
X_train, X_val, y_train, y_val = data_split_train_val(X, y)

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)


Train shape: (8000, 784)
Val shape: (2000, 784)


In [3]:
# convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()

X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val).long()

In [4]:
print("Before reshape:")
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_val_tensor shape:", X_val_tensor.shape)

Before reshape:
X_train_tensor shape: torch.Size([8000, 784])
X_val_tensor shape: torch.Size([2000, 784])


In [5]:
num_train = X_train_tensor.shape[0]
num_val = X_val_tensor.shape[0]

print(num_train, num_val)

8000 2000


In [6]:
X_train_tensor = X_train_tensor.reshape(num_train, 1, 28, 28)
X_val_tensor = X_val_tensor.reshape(num_val, 1, 28, 28)
print("X_train_tensor reshape:", X_train_tensor.shape)
print("X_val_tensor reshape:", X_val_tensor.shape)

X_train_tensor reshape: torch.Size([8000, 1, 28, 28])
X_val_tensor reshape: torch.Size([2000, 1, 28, 28])


In [7]:
class FashionMNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)

        x = self.relu(self.conv2(x))
        x = self.pool(x)

        x = torch.flatten(x, 1)   # flatten

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
data = FashionMNIST_CNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(data.parameters(), lr=0.01)

In [26]:
epochs =  21 

for epoch in range(epochs):
    optimizer.zero_grad()
    
    outputs = data(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/21, Loss: 0.3448
Epoch 2/21, Loss: 0.3481
Epoch 3/21, Loss: 0.3399
Epoch 4/21, Loss: 0.3397
Epoch 5/21, Loss: 0.3367
Epoch 6/21, Loss: 0.3310
Epoch 7/21, Loss: 0.3330
Epoch 8/21, Loss: 0.3246
Epoch 9/21, Loss: 0.3269
Epoch 10/21, Loss: 0.3214
Epoch 11/21, Loss: 0.3197
Epoch 12/21, Loss: 0.3170
Epoch 13/21, Loss: 0.3158
Epoch 14/21, Loss: 0.3116
Epoch 15/21, Loss: 0.3099
Epoch 16/21, Loss: 0.3077
Epoch 17/21, Loss: 0.3055
Epoch 18/21, Loss: 0.3033
Epoch 19/21, Loss: 0.2997
Epoch 20/21, Loss: 0.2976
Epoch 21/21, Loss: 0.2953


In [27]:
with torch.no_grad():
    outputs = data(X_val_tensor)
    predictions = torch.argmax(outputs, 1)
    correct = predictions == y_val_tensor
    correct_float = correct.float()
    accuracy = correct_float.mean().item()
    
print("Validation accuracy:", accuracy)

Validation accuracy: 0.8560000061988831
